# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/chac/Desktop/Class/Module_6_Assignment/python_api_challenge/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,melekeok village,7.4957,134.6367,303.46,70,75,3.60,PW,1706841808
1,1,port-aux-francais,-49.3500,70.2167,280.52,88,16,8.16,TF,1706841809
2,2,ushuaia,-54.8000,-68.3000,280.96,71,0,11.32,AR,1706841677
3,3,puerto natales,-51.7236,-72.4875,280.92,85,100,1.04,CL,1706841809
4,4,grytviken,-54.2811,-36.5092,276.09,71,6,4.00,GS,1706841810


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Max temp higher than 21 degrees and lower than 27 degrees
# Wind speed less than 4.5m/s
# Zero Cloudiness  
samp_data_df = city_data_df.loc[(city_data_df["Max Temp"] > 200) & (city_data_df["Max Temp"] < 300)
                 & (city_data_df["Cloudiness"]== 0),:]

# Drop any rows with null values
samp_data_df = samp_data_df.dropna()

# Display sample data
samp_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,ushuaia,-54.8000,-68.3000,280.96,71,0,11.32,AR,1706841677
9,9,khamis mushait,18.3064,42.7292,286.49,76,0,5.66,SA,1706841811
50,50,thunder bay,48.4001,-89.3168,268.14,68,0,2.06,CA,1706841713
57,57,lockhart,28.6194,-81.4426,287.64,84,0,3.09,US,1706841823
61,61,mar del plata,-38.0023,-57.5575,297.56,71,0,2.24,AR,1706841824
...,...,...,...,...,...,...,...,...,...,...
530,530,ad dabbah,18.0500,30.9500,282.55,33,0,4.08,SD,1706841954
549,549,mount gambier,-37.8333,140.7667,295.57,35,0,4.31,AU,1706841959
550,550,colac,-38.3333,143.5833,294.91,34,0,4.81,AU,1706841959
554,554,codrington,-38.2667,141.9667,292.44,52,0,6.07,AU,1706841960


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = samp_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,ushuaia,AR,-54.8000,-68.3000,71,
9,khamis mushait,SA,18.3064,42.7292,76,
50,thunder bay,CA,48.4001,-89.3168,68,
57,lockhart,US,28.6194,-81.4426,84,
61,mar del plata,AR,-38.0023,-57.5575,71,
...,...,...,...,...,...,...
530,ad dabbah,SD,18.0500,30.9500,33,
549,mount gambier,AU,-37.8333,140.7667,35,
550,colac,AU,-38.3333,143.5833,34,
554,codrington,AU,-38.2667,141.9667,52,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
khamis mushait - nearest hotel: No hotel found
thunder bay - nearest hotel: Hampton Inn & Suites by Hilton Thunder Bay
lockhart - nearest hotel: Geneva Motel
mar del plata - nearest hotel: Gran hotel Mónaco
castillos - nearest hotel: A mi gente
salto - nearest hotel: Hotel Español Salto
rio segundo - nearest hotel: Hotel Plaza
kodiak - nearest hotel: Shelikof Lodge
rio grande - nearest hotel: Hotel Vila Moura Executivo
bastrop - nearest hotel: No hotel found
illizi - nearest hotel: بوناقة
bastia - nearest hotel: Adonis Saint Florent
tegueste - nearest hotel: Hotel Aguere
punta del este - nearest hotel: Hotel Aquarium
cabo san lucas - nearest hotel: Comfort Rooms
vicuna - nearest hotel: No hotel found
nuevo ideal - nearest hotel: No hotel found
glendive - nearest hotel: La Quinta Inn & Suites by Wyndham Glendive
nouadhibou - nearest hotel: Hotel valencia
tazacorte - nearest hotel: App Leyma
wazirabad - nearest h

,City,Country,Lat,Lng,Humidity,Hotel Name
2,ushuaia,AR,-54.8000,-68.3000,71,Apart Hotel Aires del Beagle
9,khamis mushait,SA,18.3064,42.7292,76,No hotel found
50,thunder bay,CA,48.4001,-89.3168,68,Hampton Inn & Suites by Hilton Thunder Bay
57,lockhart,US,28.6194,-81.4426,84,Geneva Motel
61,mar del plata,AR,-38.0023,-57.5575,71,Gran hotel Mónaco
...,...,...,...,...,...,...
530,ad dabbah,SD,18.0500,30.9500,33,No hotel found
549,mount gambier,AU,-37.8333,140.7667,35,The Old Mount Gambier Gaol
550,colac,AU,-38.3333,143.5833,34,No hotel found
554,codrington,AU,-38.2667,141.9667,52,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)